# This notebook shows the measures of each frame of the video

In [2]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
import json
import cv2
import numpy as np
import plotly.express as px

from dash import Dash, dcc, html, Input, Output

# Import library with current code functions
sys.path.append(os.path.join("..", "lib"))
import manual_labeler_functions as man_lab_fun, general_functions as gf, files_paths as fp

## Defining some functions

In [3]:
def PLOT_MEASURE_EXPLORER(df, start_frame=None, end_frame=None):

    if start_frame==None: start_frame = df.index.min()
    if end_frame==None: end_frame = df.index.max()

    
    port = 5005
    external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
    app = Dash(__name__, external_stylesheets=external_stylesheets)

    def open_browser():
    	webbrowser.open_new("http://localhost:{}".format(port))
          
    app.layout = html.Div([
        html.H4('Measure Explorer'),
        html.H6('Reference Signal'),
        html.I('Setting the frame interval range'),
        html.Br(),
        dcc.Input(id='input-init', type='number', min=df.index.min(), max=df.index.max(), value=start_frame),
        dcc.Input(id='input-end', type='number', value=end_frame),
        dcc.Graph(id="graph"),
        dcc.Checklist(
            id="checklist",
            options=df.columns,
            value=["m1", "m3"],
            inline=True
        ),
        dcc.Store(
        id='data-output',
        data=[{
            'measures': ["m1", "m3"],
            'init_label': df.index.min(),
            'end_label': df.index.max()
        }]
        ),
        html.Br(),
        html.Details([
        html.Summary('Input Parameters'),
        dcc.Markdown(id='clientside-figure-json')
        ])
    ])
    
    @app.callback(
        Output("graph", "figure"), 
        Input("checklist", "value"),
        Input("input-init", "value"),
        Input("input-end", "value"))
    def update_line_chart(measures, frame_init, frame_end):
        mask = df.columns.isin(measures)
        filtered_df = df.loc[frame_init:frame_end, mask]
        fig = px.line(filtered_df, width=900, height=400, labels={
                     "frame_seq": "<b>Frame number</b>",
                     "value": "<b>Amplitude (pixels)</b>",
                     "variable": "<b>Measures</b>"
                 },)
        data = measures
        fig.update_layout(
            plot_bgcolor='white',
            font_family="Times Nseparate_intervalsew Roman",
            font_size=12,
        )
        fig.update_xaxes(
            mirror=True,
            ticks='outside',
            showline=True,
            linecolor='black',
            gridcolor='lightgrey',
        )
        fig.update_yaxes(
            mirror=True,
            ticks='outside',
            showline=True,
            linecolor='black',
            gridcolor='lightgrey',
        )
        return fig

    @app.callback(
        Output('data-output', 'data'),
        Input("checklist", "value"),
        Input("input-init", "value"),
        Input("input-end", "value"))
    def update_store_data(measures, frame_init, frame_end):
        mask = df.columns.isin(measures)
        filtered_df = df.loc[:,mask]
        return [{
            'measures': filtered_df.columns,
            'init_label': frame_init,
            'end_label': frame_end
        }]
    
    
    @app.callback(
        Output('clientside-figure-json', 'children'),
        Input('data-output', 'data')
    )
    def generated_data_json(data):
        return '```\n'+json.dumps(data, indent=2)+'\n```'
   
    if __name__ == '__main__':
        app.run_server(debug=True, port=port)

In [4]:
def draw_measure(img_all, measure, landmarks, color=(255, 255, 0)):
    
    # Inner mouth 1 vertical 63 and 67
    if measure == 'm1':
        image = cv2.line(img_all, (landmarks[63-1][0], landmarks[63-1][1]), 
                         (landmarks[67-1][0], landmarks[67-1][1]), color, 2, cv2.LINE_AA)
        
    # Outer mouth horizontal 49 and 55
    elif measure == 'm3':
        image = cv2.line(img_all, (landmarks[49-1][0], landmarks[49-1][1]), 
                         (landmarks[55-1][0], landmarks[55-1][1]), color, 2, cv2.LINE_AA)
        
    # Upper lips 1 vertical 52 and 63    
    elif measure == 'm4':
        image = cv2.line(img_all, (landmarks[52-1][0], landmarks[52-1][1]), 
                         (landmarks[63-1][0], landmarks[63-1][1]), color, 2, cv2.LINE_AA)
        
    # Lower lips 1 vertical 67 and 58   
    elif measure == 'm5':
        image = cv2.line(img_all, (landmarks[67-1][0], landmarks[67-1][1]), 
                         (landmarks[58-1][0], landmarks[58-1][1]), color, 2, cv2.LINE_AA)
    
    # Upper horizontal lips 62 and 64
    elif measure == 'm6':
        image = cv2.line(img_all, (landmarks[62-1][0], landmarks[62-1][1]), 
                         (landmarks[64-1][0], landmarks[64-1][1]), color, 2, cv2.LINE_AA)
        
    # Lower horizontal lips 59 and 57    
    elif measure == 'm7':
        image = cv2.line(img_all, (landmarks[59-1][0], landmarks[59-1][1]), 
                         (landmarks[57-1][0], landmarks[57-1][1]), color, 2, cv2.LINE_AA)

    # Upper lip and nose 34 and 52
    elif measure == 'm8':
        image = cv2.line(img_all, (landmarks[34-1][0], landmarks[34-1][1]), 
                         (landmarks[52-1][0], landmarks[52-1][1]), color, 2, cv2.LINE_AA)
    
    # Lower lip and nose 34 and 58
    elif measure == 'm9':
        image = cv2.line(img_all, (landmarks[34-1][0], landmarks[34-1][1]), 
                         (landmarks[58-1][0], landmarks[58-1][1]), color, 2, cv2.LINE_AA)
        
    # Inner lower lip and nose 34 and 67
    elif measure == 'm10':
        image = cv2.line(img_all, (landmarks[34-1][0], landmarks[34-1][1]), 
                         (landmarks[67-1][0], landmarks[67-1][1]), color, 2, cv2.LINE_AA)
    
    # Mouth: Draw the mean between 63 and 67 and measure vertical parallel with 34    
    elif measure == 'm11':
        mean_initial = np.array([landmarks[63-1], landmarks[67-1]]).mean(axis=0).astype("int")
        
        image_circle = cv2.circle(img_all, mean_initial, 2, color, thickness=-1)
        
        image = cv2.line(image_circle, (mean_initial[0], mean_initial[1]), 
                         (landmarks[34-1][0], landmarks[34-1][1]), color, 2, cv2.LINE_AA)
  
    # Mouth-Nose: Measure vertical parallel between 34 and 49
    elif measure == 'm12':
        image_line_1 = cv2.line(img_all, (landmarks[49-1][0], landmarks[49-1][1]), 
                         (landmarks[49-1][0], landmarks[34-1][1]), color, 2, cv2.LINE_AA)
        
        # Draw a dashed line between the coordinates
        image = draw_dashed_line(image_line_1, (landmarks[34-1][0], landmarks[34-1][1]), 
                                 (landmarks[49-1][0], landmarks[34-1][1]), color)
        
    # Mouth-Nose: Measure right vertical parallel between 34 and 55    
    elif measure == 'm13':
        image_line_1 = cv2.line(img_all, (landmarks[55-1][0], landmarks[55-1][1]), 
                         (landmarks[55-1][0], landmarks[34-1][1]), color, 2, cv2.LINE_AA)
        
        # Draw a dashed line between the coordinates
        image = draw_dashed_line(image_line_1, (landmarks[34-1][0], landmarks[34-1][1]), 
                                 (landmarks[55-1][0], landmarks[34-1][1]), color)
   
    # Mouth: Draw the mean between 49 and 55 and measure vertical parallel with 52
    elif measure == 'm14':
        mean_initial = np.array([landmarks[49-1], landmarks[55-1]]).mean(axis=0).astype("int")
        
        image_circle = cv2.circle(img_all, mean_initial, 2, color, thickness=-1)
        
        image = cv2.line(image_circle, (mean_initial[0], mean_initial[1]), 
                         (landmarks[52-1][0], landmarks[52-1][1]), color, 2, cv2.LINE_AA)
        
    # Mouth: Draw the mean between 49 and 55 and measure vertical parallel with 58
    elif measure == 'm15':
        mean_initial = np.array([landmarks[49-1], landmarks[55-1]]).mean(axis=0).astype("int")
        
        image_circle = cv2.circle(img_all, mean_initial, 2, color, thickness=-1)
        
        image = cv2.line(image_circle, (mean_initial[0], mean_initial[1]), 
                         (landmarks[58-1][0], landmarks[58-1][1]), color, 2, cv2.LINE_AA)
        
    # Mouth: Draw the mean between 49 and 55 and measure vertical parallel with 67
    elif measure == 'm16':
        mean_initial = np.array([landmarks[49-1], landmarks[55-1]]).mean(axis=0).astype("int")
        
        image_circle = cv2.circle(img_all, mean_initial, 2, color, thickness=-1)
        
        image = cv2.line(image_circle, (mean_initial[0], mean_initial[1]), 
                         (landmarks[67-1][0], landmarks[67-1][1]), color, 2, cv2.LINE_AA)
    
    # Mouth: Draw the mean between 49 and 55 and measure vertical parallel with 63
    elif measure == 'm17':
        mean_initial = np.array([landmarks[49-1], landmarks[55-1]]).mean(axis=0).astype("int")
        
        image_circle = cv2.circle(img_all, mean_initial, 2, color, thickness=-1)
        
        image = cv2.line(image_circle, (mean_initial[0], mean_initial[1]), 
                         (landmarks[63-1][0], landmarks[63-1][1]), color, 2, cv2.LINE_AA)
        
    # Left eye vertical 38 and 42
    elif measure == 'e1':
        image = cv2.line(img_all, (landmarks[38-1][0], landmarks[38-1][1]), 
                         (landmarks[42-1][0], landmarks[42-1][1]), color, 2, cv2.LINE_AA)
        
    # Right eye vertical 44 and 48
    elif measure == 'e2':
        image = cv2.line(img_all, (landmarks[44-1][0], landmarks[44-1][1]), 
                         (landmarks[48-1][0], landmarks[48-1][1]), color, 2, cv2.LINE_AA)
        
    # Eye: Vertical distance right eye2 with mean between points 45-44 and 47-48   
    elif measure == 'e3':
        mean_initial = np.array([landmarks[45-1], landmarks[44-1]]).mean(axis=0).astype("int")
        mean_final = np.array([landmarks[47-1], landmarks[48-1]]).mean(axis=0).astype("int")
        
        image_circle = cv2.circle(img_all, mean_initial, 2, color, thickness=-1)
        image_circle = cv2.circle(image_circle, mean_final, 2, color, thickness=-1)
        
        image = cv2.line(image_circle, (mean_initial[0], mean_initial[1]), 
                         (mean_final[0], mean_final[1]), color, 2, cv2.LINE_AA)
        
    # Eyebrow-Nose: Measure left vertical parallel between 28 and 20   
    elif measure == 'b1':
        image_line_1 = cv2.line(img_all, (landmarks[28-1][0], landmarks[28-1][1]), 
                         (landmarks[28-1][0], landmarks[20-1][1]), color, 2, cv2.LINE_AA)
        
        # Draw a dashed line between the coordinates
        image = draw_dashed_line(image_line_1, (landmarks[20-1][0], landmarks[20-1][1]), 
                                 (landmarks[28-1][0], landmarks[20-1][1]), color)
        
    # Eyebrow-Nose: Measure right vertical parallel between 28 and 25
    elif measure == 'b2':
        image_line_1 = cv2.line(img_all, (landmarks[28-1][0], landmarks[28-1][1]), 
                         (landmarks[28-1][0], landmarks[25-1][1]), color, 2, cv2.LINE_AA)
        
        # Draw a dashed line between the coordinates
        image = draw_dashed_line(image_line_1, (landmarks[25-1][0], landmarks[25-1][1]), 
                                 (landmarks[28-1][0], landmarks[25-1][1]), color)
    
    # Eyebrows horizontal 22 and 23  
    elif measure == 'b3':
        image = cv2.line(img_all, (landmarks[22-1][0], landmarks[22-1][1]), 
                         (landmarks[23-1][0], landmarks[23-1][1]), color, 2, cv2.LINE_AA)
    
    elif measure == 'none':
        image = img_all
    else:
        print("Set a valid measure name")
    return image


In [5]:
def draw_dashed_line(img, pt1, pt2, color=(255, 255, 0), thickness=2, gap_length=10):
    x1, y1 = pt1
    x2, y2 = pt2

    # Calculate the distance between two points
    distance = np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

    # Calculate the direction vector
    vx = (x2 - x1) / distance
    vy = (y2 - y1) / distance

    # Calculate the number of points to be plotted
    num_points = int(distance / gap_length)
    
    # Plot the points through a dashed line
    for i in range(num_points+1):
        x = int(x1 + i * gap_length * vx)
        y = int(y1 + i * gap_length * vy)
        cv2.circle(img, (x, y), thickness, color, -1)
    
    return img

In [6]:
def DISPLAY_MEASURES(frames, VD_FEATURES, measures, start_frame=None, end_frame=None, max_col=5, DISPLAY_ALL_FRAMES=False):
    
    COLLORS = [(255, 0, 0), (0, 255, 255), (0, 0, 255), (255, 255, 0), (255, 0, 255)]
    
    if DISPLAY_ALL_FRAMES:
        frames_range = frames.copy()
        landmarks_range = VD_FEATURES
    else:
        if gf.test_range(start_frame, end_frame):
            frames_range = {numero_frame: frame.copy() for numero_frame, frame in frames.items() if start_frame <= numero_frame <= end_frame}
            landmarks_range = VD_FEATURES.query("frame_seq >= @start_frame & frame_seq <= @end_frame")
        
    # Loop to display images 
    n_rows = (len(frames_range) + max_col-1) // max_col
    fig_width = 15
    fig_height  = 1 * (end_frame-start_frame+1)
    plt.figure(figsize=(fig_width, fig_height))
    for i, (frame_number, frame) in enumerate(frames_range.items()):
        landmarks = landmarks_range.iloc[i]
        refImgPts = np.array([eval(coord) for coord in landmarks[6:]])

        for ref in refImgPts:
            cv2.circle(frame, (int(ref[0]), int(ref[1])), 1, (0,255,0), thickness=1)

        for m, measure in enumerate(measures):
            draw_measure(frame, measure, refImgPts, COLLORS[m])
        
        x1 = np.min(refImgPts[:, 0])
        y1 = np.min(refImgPts[:, 1])
        x2 = np.max(refImgPts[:, 0])
        y2 = np.max(refImgPts[:, 1])
        
        frame_croped = frame[y1:y2, x1:x2]
        plt.subplot(n_rows, max_col, i+1)
        plt.imshow(frame_croped)
        plt.text(0,-10,f"frame: {frame_number}")
        plt.axis('off')
    plt.tight_layout()
    plt.show()

## Reading CSV's

In [7]:
# D for local video or Y for online video
ORIGIN = "D"
VIDEO_ID = 9

VD_INFO = gf.READ_CSV_FILE(gf.get_video_path(ORIGIN, VIDEO_ID, fp.VD_INFO))
VD_FEATURES_L1 = gf.READ_CSV_FILE(gf.get_video_path(ORIGIN, VIDEO_ID, fp.VD_FEATURES_L2))
VD_MEASURE_L0 = gf.READ_CSV_FILE(gf.get_video_path(ORIGIN, VIDEO_ID, fp.VD_MEASURE_L0))

## Getting the video frames

In [8]:
if ORIGIN == "Y":
    video_name = VD_INFO.link_video[0]
    video_frames = man_lab_fun.LOAD_VIDEO_FRAMES(gf.get_best_url(video_name))
elif ORIGIN == "D":
    VIDEO_EXT = '.mp4'
    video_name = VD_INFO.link_video[0]
    video_path = os.path.join(fp.VIDEO_SOURCE_LOCAL, VD_INFO.link_video[0] + VIDEO_EXT)
    print("video_path", video_path)
    video_frames = man_lab_fun.LOAD_VIDEO_FRAMES(str(video_path))
else: print("Invalid Origin")

video_path ../Video-Source/in_DD-Local/H9HzZV6pW7g.mp4
Error opening video


## Select the crop interval

In [9]:
start_frame_crop = 0
end_frame_crop = 5
n_frames_per_row = 4

## Selected the wanted measures to be drawn

In [10]:
# Set the measures to draw (max=5)
measures = ['m1', 'e1', 'e2', 'b1']

DISPLAY_MEASURES(video_frames, VD_MEASURE_L0, measures, start_frame_crop, end_frame_crop, n_frames_per_row)

<Figure size 1500x600 with 0 Axes>

## Plot the measure explorer graph

In [11]:
VD_MEASURE_DT = gf.READ_CSV_FILE(gf.get_video_path(ORIGIN, VIDEO_ID, fp.VD_MEASURE_L0))

# Set frames_seq as index
VD_MEASURE_DT_ED = VD_MEASURE_DT.set_index(pd.Index(VD_MEASURE_DT['frame_seq'])).copy()
VD_MEASURE_DT_ED = VD_MEASURE_DT_ED.iloc[:, 4:]

PLOT_MEASURE_EXPLORER(VD_MEASURE_DT_ED, start_frame_crop, end_frame_crop)